# Ovojnica za binarne klasifikatorje

Z ovojnico (wrapperjem) za binarne klasifikatorje želimo te uporabiti pri učenju na večrazrednih problemih. Torej na podatkih, kjer je število razredov lahko tudi večje od dva. Spodnjo ovojnico smo razvili za Orange, uporabimo pa jo lahko na kakršnemkoli učnem algoritmu Oranga, tudi na teh, ki so sicer večrazredni. Postopek, ki smo ga implementirali, je podoben postopku [one-vs-rest](http://en.wikipedia.org/wiki/Multiclass_classification), a s trikom. Da bi se izognili razredno neuravnoteženim učnim primerom, za vsak binarni klasifikator in primere, ki so uvrščeni v njegov ciljni razred, vzorčimo enako število učnih primerov iz drugih razredov. V spodnji implementaciji je še ena razlika s splošnim postopkom one-vs-rest. Napovedujemo namreč verjetnosti. Za vsak razred s klasifikatorjem za ta razred ocenimo verjetnost razreda in ga shranimo v seznam verjetnosti. Na koncu postopka seznam normaliziramo.

In [1]:
import numpy as np
import Orange
from sklearn import datasets
from matplotlib import pyplot as plt
import random
%matplotlib inline

Preberemo podatke o rožicah in preverimo, kakšno število razrednih vrednosti ima ta nabor podatkov.

In [2]:
data = Orange.data.Table("iris")
len(data.domain.class_var.values)

3

Learnerji v Orangu so objekti. Ko jim podamo podatke, nam vrnejo klasifikator, ki je prav tako objekt. Ta lahko sprejme en sam primer ali pa množico primerov. Klasifikatorji kot rezultat lahko vračajo razred (vrednost diskretne spremenljivke) ali verjetnosti razredov. Ker lahko klasifikatorje kličemo s seznamom primerov, je pri vračanju verjetnosti objekt, ki ga vrnejo, matrika. Oglejmo si to na spodnjem primeru.

In [3]:
logreg = Orange.classification.LogisticRegressionLearner()
tree = Orange.classification.SimpleTreeLearner()

In [4]:
classifier = logreg(data)
classifier(data[0]) # klasificiramo en primer, klasifikator vrne razred

Value('iris', Iris-setosa)

In [5]:
classifier(data[0], 1) # od klasifikatorja zahtevamo verjetnost razredov

array([[  8.79681649e-01,   1.20307538e-01,   1.08131372e-05]])

In [6]:
classifier(data[0:5], 1) # verjetnosti za prvih pet primerov

array([[  8.79681649e-01,   1.20307538e-01,   1.08131372e-05],
       [  7.99706325e-01,   2.00263292e-01,   3.03825365e-05],
       [  8.53796795e-01,   1.46177302e-01,   2.59031285e-05],
       [  8.25383127e-01,   1.74558937e-01,   5.79356669e-05],
       [  8.97323628e-01,   1.02665167e-01,   1.12050036e-05]])

Sledi trik, ki ga bomo potrebovali pri implementaciji naše ovojnice: iz matrike napovedanih verjetnosti bi radi pridobili verjetnosti (samo) za i-ti razred. Želeli bi ga imeti kot slopični vektor.

In [7]:
i = 1
classifier(data[0:5], 1)[:, [i]]

array([[ 0.12030754],
       [ 0.20026329],
       [ 0.1461773 ],
       [ 0.17455894],
       [ 0.10266517]])

### Ovojnica

Zdaj pa k naši ovojnici. Ker se bo ta obnašala kot Orange-ov learner, jo izvedemo iz tega razreda. V inicializacijskem delu si zapomnemo binarni learner, ki ga bo ovojnica uporabila za učenje. Potem implementiramo funkcijo fit_storage, ki je namenjena učenju, ko podatke podamo v obliki Orange-ove tabele.

In [8]:
class MulticlassLearner(Orange.classification.Learner):
    """Wraps binary learner into a multi-class learner"""
    def __init__(self, learner):
        super().__init__()
        self.learner = learner
        self.name = "mc " + learner.name
    
    def fit_storage(self, data):
        classifiers = []
        for i in range(len(data.domain.class_var.values)):
            # priprava podatkov, najprej primeri z razredom i
            X1 = data.X[data.Y == i]
            y1 = np.array([0] * X1.shape[0]) # cilji razred ima indeks 0

            # sledi vzorec primerov z razredom, ki je drugačen od i
            ind = np.arange(len(data))[data.Y != i]
            np.random.shuffle(ind)
            X0 = data.X[ind[:X1.shape[0]]]
            y0 = np.array([1] * X0.shape[0])
            
            # oblikujemo dvovrednostno domeno
            domain = Orange.data.Domain(data.domain.attributes, 
                                        Orange.data.DiscreteVariable("class", values=["T", "F"]))
            # podatke združimo v tabelo
            xy = Orange.data.Table(domain, np.vstack((X1, X0)), np.hstack((y1, y0)))
            # tabelo podamo binarnemu učnemu algoritmu in shranimo vrnjeni klasifikator v seznam
            classifiers.append(self.learner(xy))
        # seznam klasifikatorjev podamo objektu, ki bo primere klasificiral
        mc = MulticlassModel(data.domain, classifiers)
        mc.name = self.name
        return mc

Razred za klasifikacijo si ob inicializaciji zapomni seznam klasifikatorjev za posamezni razred. Implementirati mora funkcijo za napovedovanje. V naši kodi smo implementirali samo del, ki vrača verjetnosti. Tu za vsak model m pridobimo verjetnosti razredov (dvostolpična matrika) s klicem m(data, 1). Iz te matrike nato izberemo samo prvi stolpec z m(data, 1)[:, [0]]. Te stolpce, ki nam za dane testne podatke predstavljajo verjetnosti k razredov, združimo s funkcijo hstack. Dobimo matriko, kjer so stolpci ocene verjetnosti razredov, vrstice pa ustrezajo posameznim primerom v testni množici. Verjetnosti za posamezne primere moramo še normalizirati. To storimo tako, da oblikujemo vektor z vsotami po vrsticah z ps.sum(axis=1), ta vrstični vektor pa preoblikujemo v stolpčni z [:, None]. Nato vrnemo normalizirane ocene verjetnosti.

In [9]:
class MulticlassModel(Orange.classification.Model):
    """Multi-class classifier based on a set of binary classifiers."""
    def __init__(self, domain, models):
        super().__init__(domain)
        self.models = models # a list of predictors
    
    def predict(self, data, ret=1):
        """Given a data instance or table of data instances returns predicted class."""
        ps = np.hstack([m(data, 1)[:, [0]] for m in self.models])
        sums = ps.sum(axis=1)[:, None]
        return ps / sums

Sledi primer uporabe našega novega učnega postopka.

### Dela?

In [10]:
mc = MulticlassLearner(logreg)

In [11]:
cl = mc(data)

In [12]:
cl(data[10:15], 1)

array([[  8.52572914e-01,   1.47350436e-01,   7.66505579e-05],
       [  8.20845929e-01,   1.78831492e-01,   3.22579286e-04],
       [  7.55522357e-01,   2.44249363e-01,   2.28279781e-04],
       [  8.03001351e-01,   1.96720004e-01,   2.78645648e-04],
       [  8.96832364e-01,   1.03151039e-01,   1.65970916e-05]])

In [13]:
cl(data[10:15], 1).sum(axis=1) # sanity check

array([ 1.,  1.,  1.,  1.,  1.])

Dela! Čas je za pravi test. Namreč, z zgornjim smo se malo namučili po Orange-ovsko zato, da bi lahko naš učni postopek uporabljali tako, kot vse ostale učne algoritme Orange. Torej tudi v prečnem preverjanju. 

In [14]:
maj = Orange.classification.MajorityLearner()
lr = Orange.classification.LogisticRegressionLearner()
mc_lr = MulticlassLearner(lr)
mc_tree = MulticlassLearner(tree)
res = Orange.evaluation.CrossValidation(data, [mc_lr, lr])
Orange.evaluation.AUC(res)

array([ 0.965,  0.965])

Zgoraj smo za mero točnosti uporabili AUC, [površino pod krivuljo ROC](http://en.wikipedia.org/wiki/Receiver_operating_characteristic). O tej meri bo na predavanjih še govora. Pri izzivu Otto Group na [Kagglu](http://www.kaggle.com) pa kot mero uspešnosti uporabljajo [logloss](https://www.kaggle.com/wiki/LogarithmicLoss). Poskus implementacije te (preveri, če ta res daje prave vrednosti!) je spodaj. Spodnja funkcija seveda deluje nad objektom, ki ga vračajo Orange-ovi razredi za testiranje učnih algoritmov.

In [15]:
def logloss(res):
    ll = []
    for i in range(res.probabilities.shape[0]):
        # x je vektor verjetnosti, ki smo jih napovedali dejanskemu razredu
        x = np.array([v[i] for v, i in zip(res.probabilities[i], res.actual.astype(int))])
        ll.append(-sum(np.log(x))/len(x))
    return ll

In [16]:
logloss(res)

[0.39854952412347, 0.32028596555193267]